# HIDVL metadata spreadsheet merge script
As part of the process to generate draft HIDVL MARC records by batch, we need to combine metadata spreadsheets from two different sources: the legacy metadata submission form (the data from which is converted via another script from a text file to a CSV file) and the new Airtable metadata submission form.

In [1]:
#import modules and libraries
import pandas as pd
import numpy as np
from datetime import datetime, date, time
filetime = datetime.now()
filetime = filetime.strftime("%Y-%m-%d_%I-%M_%p")

## Legacy metadata
In the first part of this process, we'll deal with the legacy metadata that dates prior to 2019.

In [2]:
pre_2019_dmd = input("enter file name and if appropriate filepath of legacy metadata csv: ")

enter filepath of legacy metadata csv: hidvl_dmd_parsed_combined_2019-11-22_05-09_PM-june-december2020_pre-2019.csv


In [3]:
# load old metadata dataframe from csv
df_pre_2019_dmd = pd.read_csv(pre_2019_dmd,na_filter=False,quotechar = '"')
#print(df_pre_2019_dmd)

        HI_number     ﻿NOID Cataloged Status Publication Cycle  \
0   HI2018.057_01  vmcvdz38                     2020-08 August   
1   HI2018.058_01  qv9s4xjp                     2020-08 August   
2   HI2018.059_01  m37pvx1f                     2020-08 August   
3   HI2018.060_01  gf1vht8d                     2020-08 August   
4   HI2018.061_01  bnzs7ss7                     2020-08 August   
5   HI2018.062_01  6wwpzs8z                     2020-08 August   
6   HI2018.063_01  34tmprsw                     2020-08 August   
7   HI2018.064_01  zcrjdrdq                     2020-08 August   
8   HI2018.065_01  tmpg4qv4                     2020-08 August   
9   HI2018.066_01  pvmcvqbv                     2020-08 August   
10  HI2018.067_01  k6djhmhc                     2020-08 August   
11  HI2018.068_01  ffbg7m1g                     2020-08 August   
12  HI2018.069_01  9p8czkh0                     2020-08 August   
13  HI2018.070_01  5x69pk0v                     2020-08 August   
14  HI2018

In [4]:
#regex replaces whitespace at the end of the cells with NaN; this also has the effect of flipping "blank" cells to NaN
#I'm sure there is a more elegant way to do this.
df_pre_2019_dmd = df_pre_2019_dmd.replace(r'^\s*$', np.nan, regex=True)

In [5]:
#specify new column names for the incoming metadata column headers
pre_2019_dmd_newcols = {
    "HI_number" : "HI",
    "﻿NOID": "NOID",
    "Cataloged Status" : "Cataloged_Status",
    "Publication Cycle" : "Publication_Cycle",
    "Correction_note" : "Correction_Note",
    "Mastering Offset Timecode": "Mastering_Offset_Timecode",
    "Language": "Language_Note",
    "Subjects" : "Subjects_653",
    "Participants" : "Participants_old"

}

In [6]:
#rename the column headers
df_pre_2019_dmd.rename(columns=pre_2019_dmd_newcols, inplace=True)
#print("new df",df_pre_2019_dmd)

new df                HI      NOID  Cataloged_Status Publication_Cycle  \
0   HI2018.057_01  vmcvdz38               NaN    2020-08 August   
1   HI2018.058_01  qv9s4xjp               NaN    2020-08 August   
2   HI2018.059_01  m37pvx1f               NaN    2020-08 August   
3   HI2018.060_01  gf1vht8d               NaN    2020-08 August   
4   HI2018.061_01  bnzs7ss7               NaN    2020-08 August   
5   HI2018.062_01  6wwpzs8z               NaN    2020-08 August   
6   HI2018.063_01  34tmprsw               NaN    2020-08 August   
7   HI2018.064_01  zcrjdrdq               NaN    2020-08 August   
8   HI2018.065_01  tmpg4qv4               NaN    2020-08 August   
9   HI2018.066_01  pvmcvqbv               NaN    2020-08 August   
10  HI2018.067_01  k6djhmhc               NaN    2020-08 August   
11  HI2018.068_01  ffbg7m1g               NaN    2020-08 August   
12  HI2018.069_01  9p8czkh0               NaN    2020-08 August   
13  HI2018.070_01  5x69pk0v               NaN    2020-0

In [7]:
#add an empty column for 650 subjects
#based on https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe
df_pre_2019_dmd["Subjects_650"] = np.nan

In [8]:
#strip the period from the end of the production credits and participants fields
#based on https://stackoverflow.com/questions/37001787/remove-ends-of-string-entries-in-pandas-dataframe-column
df_pre_2019_dmd["Additional_Production_Credits"] = df_pre_2019_dmd["Additional_Production_Credits"].str.rstrip('.')
df_pre_2019_dmd["Participants_old"] = df_pre_2019_dmd["Participants_old"].str.rstrip('.')

In [9]:
#concatenate all of the participant, additional production credits, and performer columns.
#the catch is that sometimes these columns are empty!
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
df_pre_2019_dmd["Participants"] = df_pre_2019_dmd[["Additional_Production_Credits", "Participants_old", "Performers"]].apply(lambda x: '; '.join(x.dropna()), axis=1)
#print the participants field
print(df_pre_2019_dmd["Participants"])
#print a sample row to check that concatenation worked
print(df_pre_2019_dmd.loc[19,'Participants'])

0     Erick Stenflo, videographer; Andrew Boyd, perf...
1     Roland Marconi, editor; Melody Bates, Marco Ce...
2     Roland Marconi, editor; Melody Bates, Marco Ce...
3     Roland Marconi, editor; Melody Bates, Marco Ce...
4     Roland Marconi, editor; Melody Bates, Marco Ce...
5     Roland Marconi, editor; Melody Bates, Marco Ce...
6     Roland Marconi, editor; Melody Bates, Marco Ce...
7     Roland Marconi, editor; Melody Bates, Marco Ce...
8     Roland Marconi, editor; Melody Bates, Marco Ce...
9     Roland Marconi, editor; Melody Bates, Marco Ce...
10    Roland Marconi, editor; Melody Bates, Marco Ce...
11    Roland Marconi, editor; Melody Bates, Marco Ce...
12    Roland Marconi, editor; Melody Bates, Marco Ce...
13    Roland Marconi, editor; Melody Bates, Marco Ce...
14    Roland Marconi, editor; Melody Bates, Marco Ce...
15    Roland Marconi, editor; Melody Bates, Marco Ce...
16    Roland Marconi, editor; Melody Bates, Marco Ce...
17    Sarah Cameron Sunde, Maria Dizzia, associa

In [10]:
#get rid of the extra columns that we just concatenated into the new participants field
df_pre_2019_dmd.drop(["Additional_Production_Credits","Participants_old","Performers"], axis=1, inplace=True)
#get some info about the dataframe
print(df_pre_2019_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 27 columns):
HI                           20 non-null object
NOID                         20 non-null object
Cataloged_Status             0 non-null float64
Publication_Cycle            20 non-null object
Correction_Note              20 non-null object
Format                       20 non-null object
Source_Tape_Generation       20 non-null object
Mastering_Offset_Timecode    0 non-null float64
Run_Time                     20 non-null object
Series_Title                 20 non-null object
Meeting_Information          0 non-null float64
Title                        20 non-null object
Alternate_Titles             17 non-null object
Date_of_Production           17 non-null object
Location_Venue               18 non-null object
Language_Note                20 non-null object
Main_Production_Credits      19 non-null object
Worktypes                    20 non-null object
Performance_Genres           20 n

## Airtable metadata
In the second part of this process, we'll deal with the new metadata that dates from 2020 to the present.

In [11]:
post_2019_dmd = input("enter file name and if appropriate filepath of airtable metadata csv: ")

enter file name and if appropriate filepath of airtable metadata csv: Metadata-June-December 2020 publication cycle export.csv


In [12]:
# load new metadata dataframe from csv
df_post_2019_dmd = pd.read_csv(post_2019_dmd,na_filter=False,quotechar = '"')

In [13]:
df_post_2019_dmd = df_post_2019_dmd.replace(r'^\s*$', np.nan, regex=True)

In [14]:
df_post_2019_dmd = df_post_2019_dmd.fillna(np.nan)

In [15]:
# specify new column names for the incoming metadata column headers
post_2019_dmd_newcols = {
    "HI #" : "HI",
    "Inventory": "NOID",
    "Publication cycle" : "Publication_Cycle",
    "Date of event" : "Date_of_Production",
    "Location information": "Location_Venue",
    "Language note": "Language_Note",
    "Language": "Language_List",
    "Main production credits": "Main_Production_Credits",
    "Event type" : "Worktypes",
    "Subject": "Subjects_653",
    "Copyright holder": "Rights_Holder",
    "Artist bio": "Artist_Bio",
    "Run time":"Run_Time",
    "Collection": "Series_Title",
    "Conference":"Meeting_Information"
}

In [29]:
#rename the column headers
df_post_2019_dmd.rename(columns=post_2019_dmd_newcols, inplace=True)
#see what columns we have in the dataframe now:
#print("new df",df_post_2019_dmd)
print("new df",df_post_2019_dmd.info())
#Alternate titles have imported as non-null float64 values, and I'm not sure why!

new df                 HI      NOID  \
0    HI2019_034_01  ns1rnh3g   
1    HI2019_035_01  6hdr811b   
2    HI2019_036_01  rjdfn9bf   
3    HI2019_037_01  hqbzkqd1   
4    HI2019_038_01  qrfj6xj8   
5    HI2019_039_01  z8w9gr9k   
6    HI2019_040_01  c59zwb2j   
7    HI2019_041_01  pnvx0smd   
8    HI2019_042_01  h44j1612   
9    HI2019_043_01  b5mkm3vn   
10   HI2019_044_01  0zpc8dh0   
11   HI2019_045_01  2280gjfw   
12   HI2019_046_01  gtht7dvq   
13   HI2019_047_01  02v6x3zk   
14   HI2019_048_01  r7sqvj5b   
15   HI2019_049_01  98sf7tbm   
16   HI2019_050_01  d2547mn4   
17   HI2019_051_01  k0p2nprn   
18   HI2020_001_01  fxpnw4cz   
19   HI2020_002_01  4j0zpkjx   
20   HI2020_003_01  mgqnkgns   
21   HI2020_004_01  1vhhmq0t   
22   HI2020_005_01  msbcc8vb   
23   HI2020_006_01  7m0cg4z0   
24   HI2020_007_01  9kd51kgg   
25   HI2020_008_01  mgqnkhp7   
26   HI2020_009_01  8931zm3m   
27   HI2020_113_01  7d7wm9kf   
28   HI2020_114_01  4j0zpjjq   
29   HI2020_115_01  w0vt4jpm   
.

In [17]:
#add an empty column for 650 subjects
#based on https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe
df_post_2019_dmd["Subjects_650"] = np.nan

In [18]:
#do some concatenation to populate the format field
df_post_2019_dmd ["Format"] = df_post_2019_dmd["How many source media form items?"].astype(str) + " " + df_post_2019_dmd["Source media format"]

In [19]:
#combine copyright holder contact info into a single field
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
df_post_2019_dmd["Copyright_Contact"] = df_post_2019_dmd[["Copyright contact designation","Copyright address","Copyright business phone","Copyright mobile phone","Copyright fax","Copyright email 1","Copyright email 2","Copyright email 3","Copyright website"]].apply(lambda x: ', '.join(x.dropna()), axis=1)
df_post_2019_dmd["Copyright_Contact"] = df_post_2019_dmd["Copyright_Contact"].replace('\\n', ', ', regex=True)
print(df_post_2019_dmd["Copyright_Contact"])


0                                                       
1                                                       
2                                                       
3                                                       
4                                                       
5                                                       
6                                                       
7                                                       
8                                                       
9                                                       
10                                                      
11                                                      
12                                                      
13                                                      
14                                                      
15                                                      
16                                                      
17                             

In [20]:
#combine alternate titles into a single cell
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
#this may not actually concatenate any alternate titles, because there is usually only ever one
df_post_2019_dmd["Alternate_Titles"] = df_post_2019_dmd[["Alternate title 1","Alternate title 2","Alternate title 3","Alternate title 4","Alternate title 5"]].apply(lambda x: '|'.join(x.dropna()), axis=1)

In [21]:
#tried filling the blank cells in this field with np.nan but it didn't work...
df_post_2019_dmd["Alternate_Titles"].fillna(np.nan)

0                                                       
1                                                       
2      The Kidnapping of the Minister of Culture|Test...
3                                                       
4                                                       
5                                                       
6                                                       
7                                                       
8                                                       
9                                                       
10                                                      
11                                                      
12                                                      
13                                                      
14                                                      
15                                                      
16                                                      
17                             

In [22]:
#I still wanted to check to see if the values had become null values!
print(df_post_2019_dmd ["Alternate_Titles"].isnull())

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
71     False
72     False
73     False
74     False
75     False
76     False
77     False
78     False
79     False
80     False
81     False
82     False
83     False
84     False
85     False
86     False
87     False
88     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
96     False
97     False
98     False
99     False
100    False
Name: Alternate_Titles, dtype: bool


In [23]:
#get rid of any newline characters
df_post_2019_dmd["Alternate_Titles"] = df_post_2019_dmd["Alternate_Titles"].replace('\\n', '', regex=True)

In [24]:
#print a sample record that had newlines
print(df_post_2019_dmd.loc[26,"Alternate_Titles"])

A performance/presentation by Ni'Ja Whitson, an excerpted screening of Marlon Riggs’ Non, Je Ne Regrette Rien, a performance by Kiyan Williams, and a conversation moderated by Tavia Nyong’o|Test alt title


In [25]:
#drop unwanted columns and see what remains
df_post_2019_dmd.drop(["Run time rounded","DMD Finalized","How many source media form items?","Source media format","Alternate title 1","Alternate title 2","Alternate title 3","Alternate title 4","Alternate title 5","Copyright contact designation","Copyright address","Copyright business phone","Copyright mobile phone","Copyright fax","Copyright email 1","Copyright email 2","Copyright email 3","Copyright website"], axis=1, inplace=True)

In [26]:
print(df_post_2019_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 22 columns):
HI                         101 non-null object
NOID                       101 non-null object
Title                      101 non-null object
Series_Title               101 non-null object
Meeting_Information        1 non-null object
Worktypes                  101 non-null object
Date_of_Production         92 non-null object
Location_Venue             87 non-null object
Subjects_653               101 non-null object
Summary                    101 non-null object
Artist_Bio                 101 non-null object
Rights_Holder              101 non-null object
Main_Production_Credits    66 non-null object
Participants               81 non-null object
Run_Time                   100 non-null object
Language_List              101 non-null object
Language_Note              13 non-null object
Publication_Cycle          101 non-null object
Subjects_650               0 non-null float64
Format    

In [27]:
df_combined_dmd = pd.concat([df_pre_2019_dmd,df_post_2019_dmd],ignore_index=True,keys=['pre', 'post'])
#print(df_combined_dmd)
print(df_combined_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 30 columns):
Alternate_Titles             118 non-null object
Artist_Bio                   101 non-null object
Broadcast_Note               0 non-null float64
Cataloged_Status             0 non-null float64
Copyright_Contact            101 non-null object
Correction_Note              20 non-null object
Date_of_Production           109 non-null object
Format                       121 non-null object
HI                           121 non-null object
Language_List                101 non-null object
Language_Note                33 non-null object
Location_Venue               105 non-null object
Main_Production_Credits      85 non-null object
Mastering_Offset_Timecode    0 non-null float64
Meeting_Information          1 non-null object
NOID                         121 non-null object
Note_to_Cataloger            0 non-null float64
Participants                 101 non-null object
Pattern               

In [28]:
df_combined_dmd.to_csv("hidvl_metadata_combined_%s.csv"%filetime, index=False)

The end!

_Some ideas for improvement: split legacy summary column into summary and bio (currently doing this in OpenRefine)_